# Limpieza y Análisis Exploratorio de Datos (EDA)

Este notebook realiza una limpieza básica de los datos ingestados a través de la API y un análisis exploratorio de las inspecciones.

In [3]:
import pandas as pd
import boto3
import pickle
import yaml
from datetime import date


## 1. Cargar datos de S3

In [4]:
# Cargar configuración
with open("credentials.yaml", "r") as f:
    config = yaml.safe_load(f)

def cargar_datos_s3(bucket, key_prefix):
    session = boto3.Session(
        aws_access_key_id=config['s3']['aws_access_key_id'],
        aws_secret_access_key=config['s3']['aws_secret_access_key'],
        aws_session_token=config['s3']['aws_session_token']
    )
    s3 = session.client('s3')
    
    # Obtener la lista de objetos en el bucket con el prefijo dado
    response = s3.list_objects_v2(Bucket=bucket, Prefix=key_prefix)
    
    # Obtener la clave del archivo más reciente
    latest_file = max(response['Contents'], key=lambda x: x['LastModified'])['Key']
    
    # Descargar el objeto
    obj = s3.get_object(Bucket=bucket, Key=latest_file)
    
    # Cargar los datos del pickle
    dataset = pickle.loads(obj['Body'].read())
    
    return dataset

# Configurar el bucket y la clave
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key_prefix = "ingesta/inicial/"

# Cargar los datos
dataset = cargar_datos_s3(bucket, key_prefix)

print(f"Se cargaron {len(dataset)} registros de S3.")

Se cargaron 280438 registros de S3.


## 2. Transformar ingesta

In [5]:
def transformar_ingesta(dataset):
    return pd.DataFrame.from_dict(dataset)

df = transformar_ingesta(dataset)
print(f"DataFrame creado con {df.shape[0]} filas y {df.shape[1]} columnas.")
df.head()

DataFrame creado con 280438 filas y 17 columnas.


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '41.961605669949854', 'longitude'...",NaN
1,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631,2010-01-04T00:00:00.000,License,Fail,NaN,NaN,NaN,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
2,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '41.961605669949854', 'longitude'...",NaN
3,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04T00:00:00.000,Canvass,Fail,41.89843137207629,-87.6280091630558,"{'latitude': '41.89843137207629', 'longitude':...",18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
4,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.93844282365204,-87.76831838068422,"{'latitude': '41.93844282365204', 'longitude':...",NaN


## 3. Identificar faltantes

In [6]:
def faltantes(df):
    return df.isna().sum()

print("Valores faltantes por columna:")
print(faltantes(df))

Valores faltantes por columna:
inspection_id          0
dba_name               0
aka_name            2432
license_              18
facility_type       5166
risk                  83
address                0
city                 155
state                 63
zip                   41
inspection_date        0
inspection_type        1
results                0
latitude             975
longitude            975
location             975
violations         77339
dtype: int64


## 4. Eliminar faltantes de latitud y longitud

In [7]:
def elimina_faltantes_latitud_longitud(cols, df):
    return df.dropna(subset=cols)

df_limpio = elimina_faltantes_latitud_longitud(['latitude', 'longitude'], df)
print(f"Filas restantes después de eliminar faltantes: {df_limpio.shape[0]}")

Filas restantes después de eliminar faltantes: 279463


## 5. Imputar faltantes

In [8]:
def imputar_faltantes(col, value, df):
    df[col].fillna(value, inplace=True)

columns_to_impute = ['license_', 'zip', 'state', 'facility_type', 'risk']
for col in columns_to_impute:
    mode_value = df_limpio[col].mode()[0]
    imputar_faltantes(col, mode_value, df_limpio)

print("Valores faltantes después de la imputación:")
print(faltantes(df_limpio))

/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/4241019940.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(value, inplace=True)
/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/4241019940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(value, inplace=True)


Valores faltantes después de la imputación:
inspection_id          0
dba_name               0
aka_name            2419
license_               0
facility_type          0
risk                   0
address                0
city                 152
state                  0
zip                    0
inspection_date        0
inspection_type        1
results                0
latitude               0
longitude              0
location               0
violations         77064
dtype: int64


## 6. Transformación de enteros

In [9]:
def transformar_enteros(cols, df):
    for col in cols:
        df[col] = df[col].astype(int)
    return df

df_limpio = transformar_enteros(['inspection_id'], df_limpio)
print("Tipos de datos después de transformar enteros:")
print(df_limpio.dtypes)

Tipos de datos después de transformar enteros:
inspection_id       int64
dba_name           object
aka_name           object
license_           object
facility_type      object
risk               object
address            object
city               object
state              object
zip                object
inspection_date    object
inspection_type    object
results            object
latitude           object
longitude          object
location           object
violations         object
dtype: object


/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/3307818616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int)


## 7. Transformación de flotantes

In [10]:
def transformar_flotantes(cols, df):
    for col in cols:
        df[col] = df[col].astype(float)
    return df

df_limpio = transformar_flotantes(['latitude', 'longitude'], df_limpio)
print("Tipos de datos después de transformar flotantes:")
print(df_limpio.dtypes)

Tipos de datos después de transformar flotantes:
inspection_id        int64
dba_name            object
aka_name            object
license_            object
facility_type       object
risk                object
address             object
city                object
state               object
zip                 object
inspection_date     object
inspection_type     object
results             object
latitude           float64
longitude          float64
location            object
violations          object
dtype: object


/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/4228924389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/4228924389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)


## 8. Transformación de fechas

In [11]:
def transformar_fechas(cols, df):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
    return df

df_limpio = transformar_fechas(['inspection_date'], df_limpio)
print("Tipos de datos después de transformar fechas:")
print(df_limpio.dtypes)

Tipos de datos después de transformar fechas:
inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                  float64
longitude                 float64
location                   object
violations                 object
dtype: object


/var/folders/6m/hyg2m8l17n90b43ntnk_cybh0000gn/T/ipykernel_15840/2736244655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col])


## 9. Data profiling de variables categóricas

In [12]:
def data_profiling_string(cols, df):
    profiling = {}
    for col in cols:
        profiling[col] = {
            'uniques': df[col].nunique(),
            'prop_uniques': df[col].nunique() / len(df),
            'mode': df[col].mode()[0]
        }
    return pd.DataFrame(profiling)

cat_cols = ['dba_name', 'aka_name', 'license_', 'facility_type', 'risk', 'city', 'state']
print("Data profiling de variables categóricas:")
print(data_profiling_string(cat_cols, df_limpio))

Data profiling de variables categóricas:
             dba_name  aka_name  license_ facility_type           risk  \
uniques         32809     31210     45534           513              4   
prop_uniques   0.1174  0.111678  0.162934      0.001836       0.000014   
mode           SUBWAY    SUBWAY         0    Restaurant  Risk 1 (High)   

                  city     state  
uniques             23         1  
prop_uniques  0.000082  0.000004  
mode           CHICAGO        IL  


## 10. Data profiling de fechas

In [13]:
def data_profiling_fechas(cols, df):
    profiling = {}
    for col in cols:
        profiling[col] = {
            'uniques': df[col].nunique(),
            'prop_uniques': df[col].nunique() / len(df),
            'mode': df[col].mode()[0],
            'min_date': df[col].min(),
            'max_date': df[col].max(),
            'unique_years': df[col].dt.year.nunique(),
            'total_days': (df[col].max() - df[col].min()).days
        }
    return pd.DataFrame(profiling)

print("Data profiling de fechas:")
print(data_profiling_fechas(['inspection_date'], df_limpio))

Data profiling de fechas:
                  inspection_date
uniques                      3730
prop_uniques             0.013347
mode          2013-11-14 00:00:00
min_date      2010-01-04 00:00:00
max_date      2024-10-17 00:00:00
unique_years                   15
total_days                   5400


## 11. Guardar datos limpios en S3

In [21]:
import boto3
import pickle
from datetime import date
from botocore.exceptions import ClientError

# Se agregan las credenciales ya que al no ser un .py y ser cuenta de estudiante constantemente se deben refrescar

# Configurar el cliente de boto3 con las credenciales
boto3.setup_default_session(
    aws_access_key_id=config['s3']['aws_access_key_id'],
    aws_secret_access_key=config['s3']['aws_secret_access_key'],
    aws_session_token=config['s3']['aws_session_token']
)

# Definir la región
AWS_REGION = "us-east-1"

def guardar_datos_s3(df):
    bucket = f"aplicaciones-cd-1-{config['iexe']['matricula']}"
    today = date.today().strftime("%Y-%m-%d")
    key = f"limpieza/datos-limpios-{today}.pkl"
    
    s3 = boto3.resource('s3', region_name=AWS_REGION)
    
    try:
        # Serializar el DataFrame
        pickle_data = pickle.dumps(df)
        
        # Subir los datos a S3
        s3.Object(bucket, key).put(Body=pickle_data)
        print(f"Datos guardados exitosamente en {bucket}/{key}")
        return True
    except ClientError as e:
        print(f"Error al guardar los datos: {e}")
        return False

# ejecucion principal
if 'df_limpio' in globals():
    if guardar_datos_s3(df_limpio):
        print("Proceso de guardado completado con éxito.")
else:
    print("Error: 'df_limpio' no está definido. Asegúrate de tener datos limpios para guardar.")

print(f"Región AWS utilizada: {AWS_REGION}")

Datos guardados exitosamente en aplicaciones-cd-1-mcda24a004/limpieza/datos-limpios-2024-10-20.pkl
Proceso de guardado completado con éxito.
Región AWS utilizada: us-east-1


## 12Escenario de limpieza EXTRA o PLUS que no se agrega a la carga para no modificar el resultado esperado y se incluye para demostrar conocimeinto y entendimeinto del probelma

In [22]:
# 12. Detección y manejo de valores atípicos

def detectar_y_manejar_atipicos(df, columnas_numericas, metodo='zscore', umbral=3):
    """
    Detecta y maneja valores atípicos en las columnas numéricas especificadas.
    
    Args:
    df (pandas.DataFrame): El DataFrame a procesar.
    columnas_numericas (list): Lista de columnas numéricas para analizar.
    metodo (str): Método para detectar atípicos ('zscore' o 'iqr').
    umbral (float): Umbral para considerar un valor como atípico.
    
    Returns:
    pandas.DataFrame: DataFrame con los valores atípicos manejados.
    """
    import numpy as np
    from scipy import stats
    
    df_limpio = df.copy()
    
    for columna in columnas_numericas:
        if metodo == 'zscore':
            # Método Z-score
            z_scores = np.abs(stats.zscore(df_limpio[columna]))
            mask = z_scores > umbral
        elif metodo == 'iqr':
            # Método IQR (Rango Intercuartil)
            Q1 = df_limpio[columna].quantile(0.25)
            Q3 = df_limpio[columna].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - umbral * IQR
            upper_bound = Q3 + umbral * IQR
            mask = (df_limpio[columna] < lower_bound) | (df_limpio[columna] > upper_bound)
        else:
            raise ValueError("Método no reconocido. Use 'zscore' o 'iqr'.")
        
        # Conteo de valores atípicos
        n_outliers = mask.sum()
        print(f"Se detectaron {n_outliers} valores atípicos en la columna {columna}")
        
        # Manejar valores atípicos (en este caso, los reemplazamos por la mediana)
        df_limpio.loc[mask, columna] = df_limpio[columna].median()
    
    return df_limpio

# Aplicar la función a nuestro DataFrame
columnas_numericas = ['latitude', 'longitude']  # Añadir otras columnas numéricas según sea necesario
df_sin_atipicos = detectar_y_manejar_atipicos(df_limpio, columnas_numericas)

print("Resumen estadístico después de manejar valores atípicos:")
print(df_sin_atipicos[columnas_numericas].describe())

# Comparación antes y después
for col in columnas_numericas:
    print(f"\nComparación de {col}:")
    print("Antes:")
    print(df_limpio[col].describe())
    print("Después:")
    print(df_sin_atipicos[col].describe())

"""
Justificación:
1. Detección de valores atípicos: Es crucial identificar y manejar valores atípicos 
   ya que pueden distorsionar significativamente los análisis estadísticos y los modelos predictivos.

2. Flexibilidad en el método: Ofrecemos dos métodos comunes (Z-score e IQR) para 
   detectar atípicos, permitiendo adaptarse a diferentes distribuciones de datos.

3. Transparencia: Informamos sobre la cantidad de valores atípicos detectados, 
   lo que ayuda a entender mejor la calidad de los datos.

4. Manejo conservador: Reemplazamos los valores atípicos por la mediana de la columna, 
   lo cual es una estrategia conservadora que preserva la distribución general de los datos.

5. Comparación antes/después: Mostramos estadísticas descriptivas antes y después del 
   tratamiento, lo que permite evaluar el impacto de la limpieza.

6. Mejora en la calidad de datos: Al manejar los valores atípicos, mejoramos la 
   calidad general de los datos para análisis posteriores y modelado.

Esta adición complementa el trabajo de limpieza ya realizado, abordando un aspecto 
importante que no se había cubierto previamente y que es una práctica común en 
preparación de datos para análisis y modelado en ciencia de datos.
"""

Se detectaron 0 valores atípicos en la columna latitude
Se detectaron 3542 valores atípicos en la columna longitude
Resumen estadístico después de manejar valores atípicos:
            latitude      longitude
count  279463.000000  279463.000000
mean       41.880622     -87.673153
std         0.081011       0.052173
min        41.644670     -87.846320
25%        41.831542     -87.704734
50%        41.891733     -87.666126
75%        41.939713     -87.634827
max        42.021064     -87.525094

Comparación de latitude:
Antes:
count    279463.000000
mean         41.880622
std           0.081011
min          41.644670
25%          41.831542
50%          41.891733
75%          41.939713
max          42.021064
Name: latitude, dtype: float64
Después:
count    279463.000000
mean         41.880622
std           0.081011
min          41.644670
25%          41.831542
50%          41.891733
75%          41.939713
max          42.021064
Name: latitude, dtype: float64

Comparación de longitude:
Ante

'\nJustificación:\n1. Detección de valores atípicos: Es crucial identificar y manejar valores atípicos \n   ya que pueden distorsionar significativamente los análisis estadísticos y los modelos predictivos.\n\n2. Flexibilidad en el método: Ofrecemos dos métodos comunes (Z-score e IQR) para \n   detectar atípicos, permitiendo adaptarse a diferentes distribuciones de datos.\n\n3. Transparencia: Informamos sobre la cantidad de valores atípicos detectados, \n   lo que ayuda a entender mejor la calidad de los datos.\n\n4. Manejo conservador: Reemplazamos los valores atípicos por la mediana de la columna, \n   lo cual es una estrategia conservadora que preserva la distribución general de los datos.\n\n5. Comparación antes/después: Mostramos estadísticas descriptivas antes y después del \n   tratamiento, lo que permite evaluar el impacto de la limpieza.\n\n6. Mejora en la calidad de datos: Al manejar los valores atípicos, mejoramos la \n   calidad general de los datos para análisis posteriore